1. From TF graph, decide wich tensors you want to log
```python
with tf.name_scope(layer1) as scope:
    tf.summary.image()
    tf.summary.histogram()
    tf.summary.scalar()
```
2. Merge all summaries
```python
tf.summary.merge_all()
```
3. Create writer and add graph
```python
writer = tf.summary.FileWriter("./logs/example")
writer.add_graph(sess.graph)
```
4. Run summary merge and add_summary
```python
s, _ = sess.run([summary, optimizer], feed_dict=feed_dict)
writer.add_summary(s, global_step=global_step)
```
5. Launch TensorBoard
```
tensorboard --logdir=/logs/example
```

In [1]:
import tensorflow as tf
import numpy as np

learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)


with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                           tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor")
    writer.add_graph(sess.graph)

    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={
                  X: x_data, Y: y_data}), sess.run([W1, W2]))

    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)


0 0.9352565 [array([[-0.46451557, -1.82273   ],
       [ 2.2659514 ,  0.3523676 ]], dtype=float32), array([[0.41778436],
       [0.7280225 ]], dtype=float32)]
100 0.691381 [array([[-0.45754576, -2.2297792 ],
       [ 1.9669846 ,  0.15833347]], dtype=float32), array([[-0.17989142],
       [ 0.16853626]], dtype=float32)]
200 0.67403716 [array([[-1.991351 , -2.3075728],
       [ 3.1721303,  0.5252328]], dtype=float32), array([[-0.3843168 ],
       [ 0.25501955]], dtype=float32)]
300 0.6080351 [array([[-3.615977 , -2.8287826],
       [ 4.7616024,  1.3029305]], dtype=float32), array([[-0.89964753],
       [ 0.713569  ]], dtype=float32)]
400 0.4449249 [array([[-5.064114 , -4.2082987],
       [ 6.0589023,  2.930019 ]], dtype=float32), array([[-1.6402358],
       [ 1.7868849]], dtype=float32)]
500 0.2898146 [array([[-6.1137247, -5.4029903],
       [ 7.038144 ,  4.376981 ]], dtype=float32), array([[-2.420085 ],
       [ 2.9648244]], dtype=float32)]
600 0.20082986 [array([[-6.8519726, -6.1943536

5400 0.0019255774 [array([[-11.774778, -10.936493],
       [ 12.74095 ,  10.127653]], dtype=float32), array([[-12.331911],
       [ 13.099305]], dtype=float32)]
5500 0.0018219089 [array([[-11.809741, -10.970489],
       [ 12.776397,  10.161366]], dtype=float32), array([[-12.441883 ],
       [ 13.2085495]], dtype=float32)]
5600 0.0017242684 [array([[-11.844233, -11.004036],
       [ 12.811364,  10.194633]], dtype=float32), array([[-12.551266],
       [ 13.317222]], dtype=float32)]
5700 0.0016323704 [array([[-11.878271, -11.037149],
       [ 12.845871,  10.227461]], dtype=float32), array([[-12.660093],
       [ 13.425352]], dtype=float32)]
5800 0.0015457354 [array([[-11.911871, -11.069852],
       [ 12.879932,  10.25988 ]], dtype=float32), array([[-12.768392],
       [ 13.532976]], dtype=float32)]
5900 0.0014640784 [array([[-11.945055, -11.102156],
       [ 12.913572,  10.291902]], dtype=float32), array([[-12.876192],
       [ 13.640119]], dtype=float32)]
6000 0.0013870997 [array([[-11.9